In [2]:
import os
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


class CoronarySmallDataset(Dataset):
    def __init__(self, image_dir, keypoint_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.keypoint_dir = keypoint_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.input = os.listdir(image_dir)

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        SIZE = 256
        img_path = os.path.join(self.image_dir, self.input[idx])
        keypoint_path = os.path.join(self.keypoint_dir, self.input[idx])
        mask_path = os.path.join(self.mask_dir, self.input[idx])
        

        image = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
        image = cv2.resize(image, (SIZE, SIZE))

        keypoint = cv2.imread(keypoint_path, cv2.IMREAD_UNCHANGED)
        keypoint = cv2.resize(keypoint, (SIZE, SIZE), interpolation=cv2.INTER_NEAREST)
        
        keypoint =  np.expand_dims(keypoint, axis=2)
        output = np.concatenate((image, keypoint), axis=2)

        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        mask = cv2.resize(mask, (SIZE, SIZE), interpolation=cv2.INTER_NEAREST)
        
        if self.transform:
            output = self.transform(output)
            mask = torch.tensor(mask, dtype=torch.long)
        
        return output, mask


transform = transforms.Compose([
    transforms.ToTensor()
])

train_image_dir = '../images/images_left/images_train/input_dicom'
train_keypoint_dir = '../images/images_left/images_train/keypoints'
train_mask_dir = '../images/images_left/images_train/output'
val_image_dir = '../images/images_left/images_val/input_dicom'
val_keypoint_dir = '../images/images_left/images_val/keypoints'
val_mask_dir = '../images/images_left/images_val/output'
test_image_dir = '../images/images_left/images_test/input_dicom'
test_keypoint_dir = '../images/images_left/images_test/keypoints'
test_mask_dir = '../images/images_left/images_test/output'

train_dataset = CoronarySmallDataset(train_image_dir, train_keypoint_dir, train_mask_dir, transform=transform)
val_dataset = CoronarySmallDataset(val_image_dir, val_keypoint_dir, val_mask_dir, transform=transform)
test_dataset = CoronarySmallDataset(test_image_dir, test_keypoint_dir, test_mask_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=6, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=6, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=6, shuffle=False)


In [3]:
from converter_RGB import convert_int_to_RGB
from large_RGB_model import UNet


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet()
model = model.to(device)

model.load_state_dict(torch.load('model_dicom_kp_left_1.pth', map_location=device))
model.eval()

def show_image(type, image_name):
    dir = f"..\images\images_test\{type}"
    print(dir)
    if type == 'output':
        img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
        # img = img[:, :, 2]
        print("Oryginalnie: ", np.unique(img))
        
        # print("Po resize: NEAREST\t  ", np.unique(cv2.resize(img, (256, 256), interpolation=cv2.INTER_NEAREST)))
        # cv2.imshow('INTER_NEAREST', cv2.resize(convert_int_to_RGB(cv2.resize(img, (256, 256), interpolation=cv2.INTER_NEAREST)), (512, 512)))
        # print("Po resize: NEAREST_EXACT  ", np.unique(cv2.resize(img, (256, 256), interpolation=cv2.INTER_NEAREST_EXACT)))
        # cv2.imshow('INTER_NEAREST_EXACT', cv2.resize(convert_int_to_RGB(cv2.resize(img, (256, 256), interpolation=cv2.INTER_NEAREST_EXACT)), (512, 512)))
        
        print("Po resize:  ", np.unique(cv2.resize(img, (256, 256))))
        # print(img)
        img = convert_int_to_RGB(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    else:
        img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
        # img = img * 7
    # img = cv2.resize(img, (256, 256))
    # np.set_printoptions(threshold=np.inf)
    cv2.imshow(type, img)

def predict(image_name):
    SIZE = 256
    dir = '..\images\images_test\input_dicom'
    img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
    img = cv2.resize(img, (SIZE, SIZE))
    
    keypoint_path ='..\images\images_test\keypoints'
    keypoint = cv2.imread(os.path.join(keypoint_path, image_name), cv2.IMREAD_UNCHANGED)
    keypoint = cv2.resize(keypoint, (SIZE, SIZE), interpolation=cv2.INTER_NEAREST)
    
    keypoint =  np.expand_dims(keypoint, axis=2)
    output = np.concatenate((img, keypoint), axis=2)
    
    output = transforms.ToTensor()(output).unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model.predict(output)
        pred = pred.squeeze().cpu().numpy()
    print("Z sieci:    ", np.unique(pred))

    pred_image = convert_int_to_RGB(pred)
    pred_image = cv2.resize(pred_image, (512, 512))
    pred_image = cv2.cvtColor(pred_image, cv2.COLOR_BGR2RGB)
    cv2.imshow('pred', pred_image)
    cv2.waitKey(0)
   

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_35788\1062337145.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_kp_left_1.pt

In [10]:
# image_name = "131aedfhs6pnf1fvtvp49mhdb2fucqzo22_29.png"
image_name = "131aedfhs6pnf1fvtvp49mld7mqexnz322_36.png"
# image_name = "131aedfhs6pnf1fvtvp49mia892s56cf22_28.png"
# image_name = "131aedfhs6pnf1fvtvp49juwu7plj9dv22_40.png"
# image_name = "13c2ur549vohc0jat2dvu3xs7q1_18.png"
# image_name = "131aedfhs6pnf1fvtvp49mi8hml81goj22_37.png"

# treningowe
# image_name = "13c2ur549vohc0jat2e6y42xk11_31.png"
# image_name = "131aedfhs6pnf1fvtvp49jstof8iams322_36.png"
# image_name = "131aedfhs6pnf1fvtvp49mff2plx3rdw22_24.png"
show_image("input_dicom", image_name)
show_image("output", image_name)
predict(image_name)

..\images\images_test\input_dicom
..\images\images_test\output
Oryginalnie:  [ 0  5  6  7  8 13 15 17 18]
Po resize:   [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
Z sieci:     [ 0  2  3  4  5  6  7  8  9 13 15 17 18 22]


In [4]:
from sklearn.metrics import precision_score, recall_score, f1_score


def compute_accuracy(model, data_loader, device):
    model.eval()
    correct = 0
    correct_clear = 0
    total = 0
    total_clear = 0

    with torch.no_grad():
        for images, masks in data_loader:
            images = images.to(device)
            masks = masks.to(device)

            preds = model.predict(images)
            
            correct += (preds == masks).sum().item()
            correct_clear += (preds[masks != 0] == masks[masks != 0]).sum().item()
            total += masks.numel()
            total_clear += masks[masks != 0].numel()

    accuracy = correct / total
    accuracy_clear = correct_clear / total_clear
    return accuracy, accuracy_clear

In [5]:
def compute_precision_recall_f1(model, data_loader, device, num_classes=28):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, masks in data_loader:
            images = images.to(device)
            masks = masks.to(device)

            preds = model.predict(images)
            
            all_preds.append(preds.view(-1).cpu().numpy())
            all_targets.append(masks.view(-1).cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)

    TP = np.zeros(num_classes)
    FP = np.zeros(num_classes)
    FN = np.zeros(num_classes)

    for i in range(num_classes):
        TP[i] = ((all_preds == i) & (all_targets == i)).sum()
        FP[i] = ((all_preds == i) & (all_targets != i)).sum()
        FN[i] = ((all_preds != i) & (all_targets == i)).sum()
    
    TP[0], FP[0], FN[0] = 0, 0, 0
    # print(TP, FP, FN)

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precision * recall / (precision + recall)
    # print(TP.sum(), FP.sum(), FN.sum())

    precision_weighted = TP.sum() / (TP.sum() + FP.sum())
    recall_weighted = TP.sum() / (TP.sum() + FN.sum())
    f1_weighted = 2 * precision_weighted * recall_weighted / (precision_weighted + recall_weighted)

    return precision, recall, f1, precision_weighted, recall_weighted, f1_weighted

In [6]:
# learning_rate = 2.4, dropout_rate = 0.2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_dicom_kp_left_1.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_35788\1136538210.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_kp_left_1.pt

Train Accuracy: 97.18%
Train Accuracy Clear: 60.50%

Validation Accuracy: 96.39%
Validation Accuracy Clear: 48.56%


In [7]:
model.load_state_dict(torch.load('model_dicom_kp_left_1.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_35788\2960608092.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_kp_left_1.pt

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 2: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 3: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 4: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 5: Precision: 79.79%, Recall: 85.19%, F1: 82.40%
Class 6: Precision: 64.47%, Recall: 64.19%, F1: 64.33%
Class 7: Precision: 64.86%, Recall: 60.11%, F1: 62.40%
Class 8: Precision: 62.20%, Recall: 57.11%, F1: 59.55%
Class 9: Precision: 34.79%, Recall: 53.93%, F1: 42.30%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 58.67%, Recall: 60.24%, F1: 59.45%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 39.52%, Recall: 54.40%, F1: 45.78%
Class 16: Precision: 12.52%, Recall: 9.05%, F1: 10.51%
Class 17: Precision: 58.49%, Recall: 63.52%, F1: 60.90%
Class 18: Precision: 17.82%, R

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_35788\1128418758.py:31: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_35788\1128418758.py:32: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_35788\1128418758.py:33: RuntimeWarning: invalid value encountered in divide
  f1 = 2 * precision * recall / (precision + recall)


In [8]:
# learning_rate = , dropout_rate = 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_dicom_kp_left_2.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_35788\2273074047.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_kp_left_2.pt

Train Accuracy: 96.66%
Train Accuracy Clear: 51.98%

Validation Accuracy: 96.17%
Validation Accuracy Clear: 44.86%


In [9]:
model.load_state_dict(torch.load('model_dicom_kp_left_2.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_35788\3361012915.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_kp_left_2.pt

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 2: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 3: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 4: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 5: Precision: 75.12%, Recall: 84.43%, F1: 79.50%
Class 6: Precision: 52.83%, Recall: 68.53%, F1: 59.67%
Class 7: Precision: 62.26%, Recall: 53.97%, F1: 57.82%
Class 8: Precision: 51.58%, Recall: 65.47%, F1: 57.70%
Class 9: Precision: 32.44%, Recall: 42.33%, F1: 36.73%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 55.05%, Recall: 44.71%, F1: 49.34%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 32.04%, Recall: 48.51%, F1: 38.59%
Class 16: Precision: 6.09%, Recall: 0.06%, F1: 0.12%
Class 17: Precision: 53.77%, Recall: 55.77%, F1: 54.75%
Class 18: Precision: 33.82%, Rec

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_35788\1128418758.py:31: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_35788\1128418758.py:32: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)


In [10]:
# learning_rate = 2.4, dropout_rate = 0.3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_dicom_kp_left_3.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_35788\260049013.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_kp_left_3.pth

Train Accuracy: 97.57%
Train Accuracy Clear: 68.51%

Validation Accuracy: 96.41%
Validation Accuracy Clear: 50.39%


In [11]:
model.load_state_dict(torch.load('model_dicom_kp_left_3.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_35788\3537564718.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_kp_left_3.pt

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 2: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 3: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 4: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 5: Precision: 79.75%, Recall: 83.96%, F1: 81.80%
Class 6: Precision: 69.00%, Recall: 62.59%, F1: 65.64%
Class 7: Precision: 58.42%, Recall: 66.10%, F1: 62.03%
Class 8: Precision: 62.10%, Recall: 58.44%, F1: 60.22%
Class 9: Precision: 36.45%, Recall: 57.47%, F1: 44.61%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 57.06%, Recall: 65.24%, F1: 60.88%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 38.25%, Recall: 60.89%, F1: 46.98%
Class 16: Precision: 16.20%, Recall: 0.20%, F1: 0.40%
Class 17: Precision: 59.15%, Recall: 61.24%, F1: 60.18%
Class 18: Precision: 25.46%, Re

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_35788\1128418758.py:31: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_35788\1128418758.py:32: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)
